<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Kaggle_Contradictory%2C_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contradictory, My Dear Watson

Detecting contradiction and entailment in multilingual text **using TPUs**

- TPU : 구글에서 2016년 5월에 발표한 데이터 분석 및 딥러닝용 하드웨어이다. 벡터/행렬연산의 병렬처리에 특화되어 있으며 넘사벽급의 전성비를 자랑한다.

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers
!pip install sentencepiece

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  34567      0 --:--:-- --:--:-- --:--:-- 34567
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.8 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

# Bert model 

---



==> Bidirectional(양방향 모델) : 왼쪽에서 오른쪽으로, 오른쪽에서 왼쪽으로 모두 텍스트를 읽을 수 있음을 의미합니다.

In [3]:
# Bert Vocabulary

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
len(tokenizer.vocab)

30522

In [5]:
# token의 구성
bert_vocab = tokenizer.vocab

print(bert_vocab['[CLS]'])
print(bert_vocab['[SEP]'])
print(bert_vocab['[PAD]']) # padding=0

# 문자열 토큰화
print(bert_vocab['hello'])
print(bert_vocab['world'])

bert_keys = []

for token in tokenizer.vocab.keys():
  bert_keys.append(token)

# bert_keys = dict([(j,i) for (i,j) in bert_vocab.items()])

print(bert_keys[101])
print(bert_keys[102])
print(bert_keys[0])

print(bert_keys[7592])
print(bert_keys[2088])

101
102
0
7592
2088
[CLS]
[SEP]
[PAD]
hello
world


In [ ]:
# Bert model 구성요소

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased',
                                                      num_labels=3,  # 결과 라벨 수
                                                      output_attentions=False,
                                                      output_hidden_states=False)

outputs = model(input_ids=b_input_ids,
                token_type_ids=b_token_type_ids,
                attention_mask=b_attention_mask,
                labels=b_labels)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

NameError: ignored

1. input_ids
 - 토큰으로 표시되는 문장(토큰)
 
 [CLS] : classifier token, vlaue:101

 [SEP] : separator token, value:102

 [PAD] : padding token, value:0

  For one sentence as input:
    - [CLS] ...word tokens... [SEP]

  For two sentences as input:
    - [CLS] ...sentence1 tokens... [SEP]..sentence2 tokens... [SEP]

ex) [101[CLS], 7592, 2045, 1012,  102[SEP],    0,    0,    0,    0,    0]

2. token_type_ids
 - 입력의 일부가 되어야 하는 두 문장이 있을 때 사용
 - [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
 - 0은 첫번째 sentence 식별
 - 1은 두번째 sentence 식별
 - 오른쪽 0은 padding

3. attantion_mask
 - attention_mask의 길이는 input_ids의 길이와 같아야 한다.
 - 모델에 input_ids의 어떤 토큰이 워드이고 어떤 토큰이 패딩인지 알려준다.

ex)
 - Tokens: [101, 7592, 2045, 1012,  102,    0,    0,    0,    0,    0]
 - Attention mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

4. labels
 - 각 행의 label(target)
 - ex) [0, 2, 0, 1, 2, 0, 3, 1]

---

# how to use tokenizer to auto format input data

In [12]:
# encode_plus 실습1
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')  # multilingual tokenizer

max_len = 64
sentence1 = '나의 살던 고향은 꽃피는 동산'

encoded_dict = tokenizer.encode_plus(sentence1,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')

decode = tokenizer.decode(encoded_dict['input_ids'][0],
                          skip_special_tokens=False)
decode

'[CLS] 나의 살던 고향은 꽃피는 동산 [SEP]'

In [ ]:
# encode
max_length = 10
sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(
    sentence1,  # text to tokens
    add_special_tokens=True,  # [CLS], [SEP]를 추가
    max_length=max_length,
    truncation=True,  # # Pad or truncate.
    return_attention_mask=True,
    return_tensors='pt'
)

encoded_dict

{'input_ids': tensor([[ 101, 7592, 2045, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [13]:
input_ids = encoded_dict['input_ids'][0]
token_type_ids = encoded_dict['token_type_ids'][0]
attention_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(token_type_ids)
print(attention_mask)

tensor([  101,  1165, 25539, 10576,  1172, 82720, 34606, 47468, 75301, 11375,
         1164, 29347, 70103, 90341, 11192,  1166, 82890, 22214,   102])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [14]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

[CLS] 나의 살던 고향은 꽃피는 동산 [SEP]
나의 살던 고향은 꽃피는 동산


# XLM-RoBERTa Vocabulary

- XLM은 교차 언어 모델을 의미. 

- XLM-Roberta(XLM-R)는 다국어 BERT를 능가하는 사전 훈련된 **다국어 모델**이다.
 - XLM-R이 훨씬 더 많은 데이터를 사용하여 교육을 받았기 때문입니다. (XLM-R은 또한 100개의 언어로 train 되었다.)

1. input_ids
 - BERT의 토큰화와 비슷
    - bos_token_id (s) : 0
    - eos_token_id (/s) : 2
    - sep_token_id (/s) : 2
    - pad_token_id (pad) : 1

 - For one sentence as input:
    - (s) ...word tokens... (/s)

 - For two sentences as input:<br>
    - (s) ...sentence1 tokens... (/s)(/s)..sentence2 tokens... (/s)

 - padding은 1
    - ex)[[0, 35378, 2685, 5, 2, 1, 1, 1, 1, 1]]


2. token_type_ids
 - XLM-RoBERTa는 token_type_ids를 사용하지 않는다.

3. attention_mask
 - BERT와 같다.

4. labels
 - target 레이블(BERT와 같다.)


In [3]:
# xlm-roberta 실습1

max_len = 512

sentence1 = '코로나19 예방접종 대응 추진단은 3일 오전 10시 20분 기준으로 잠정집계한 결과 코로나19 예방접종을 1회 이상 실시한 사람이 2,000만명을 넘었다고 밝혔다. 3일 10시 20분 현재 누적 1차 접종자는 2,000만4,714명으로 전 국민의 39.0%에 해당되며 이 중 721만6,679명이 접종을 완료했다. 추진단은 “전문가와 정부를 믿고 예방접종에 적극 동참한 국민들과 예방접종을 안전하게 시행해준 전국 위탁의료기관, 예방접종센터, 보건소 의료진과 실무자 덕분”이라고 밝혔다. 추진단은 “백신 수급 관리 등 철저한 준비를 통해 9월 중 3,600만 명 이상 1차 접종 목표 조기달성에 최선을 다할 것”이라고 덧붙였다.'

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
encoded_dict = tokenizer.encode_plus(sentence1,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')
a = tokenizer.decode(encoded_dict['input_ids'][0], skip_special_tokens=False)
a

'<s> 코로나19 예방접종 대응 추진단은 3일 오전 10시 20분 기준으로 잠정집계한 결과 코로나19 예방접종을 1회 이상 실시한 사람이 2,000만명을 넘었다고 밝혔다. 3일 10시 20분 현재 누적 1차 접종자는 2,000만4,714명으로 전 국민의 39.0%에 해당되며 이 중 721만6,679명이 접종을 완료했다. 추진단은 “전문가와 정부를 믿고 예방접종에 적극 동참한 국민들과 예방접종을 안전하게 시행해준 전국 위탁의료기관, 예방접종센터, 보건소 의료진과 실무자 덕분”이라고 밝혔다. 추진단은 “백신 수급 관리 등 철저한 준비를 통해 9월 중 3,600만 명 이상 1차 접종 목표 조기달성에 최선을 다할 것”이라고 덧붙였다.</s>'

In [4]:
model_type='xlm-roberta-large'
tokenizer = XLMRobertaTokenizer.from_pretrained(model_type)

In [ ]:
print(tokenizer.vocab_size)  # check vocab size
tokenizer.special_tokens_map # what are the special tokens

250002


{'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
# xlm-roberta-base model

model_type = 'xlm-roberta-base'

model = XLMRobertaForSequenceClassification.from_pretrained(model_type,
                                                            num_labels=3)

In [ ]:
max_len = 10

text = '너의 이름은?'

encoded_dict = tokenizer.encode_plus(text,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')
encoded_dict

{'input_ids': tensor([[    0, 70204,   367, 50932,   697,    32,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
decode = tokenizer.decode(encoded_dict['input_ids'][0],
                          skip_special_tokens=False)

print(f"토큰화 된 것:{encoded_dict['input_ids'][0]}")
print(f'토큰화 안된것(디코드 한 것):{decode}')

토큰화 된 것:tensor([    0, 70204,   367, 50932,   697,    32,     2])
토큰화 안된것(디코드 한 것):<s> 너의 이름은?</s>


In [ ]:
# encode
max_len = 10

sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(sentence1,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')

encoded_dict
# input_ids, attention_mask로 구성 -- Bert는 input_ids, attention_mask, token_type_ids 구성

{'input_ids': tensor([[    0, 33600,    31,  2685,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(att_mask)

tensor([    0, 33600,    31,  2685,     5,     2])
tensor([1, 1, 1, 1, 1, 1])


In [ ]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)  # 'bos_token': '<s>', 'cls_token': '<s>', 'eos_token': '</s>', 'mask_token': '<mask>', 'pad_token': '<pad>', 'sep_token': '</s>', 'unk_token': '<unk>' 등

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

<s> hello there.</s>
hello there.


# Contradictory, My Dear Watson 실습 Start

In [2]:
train = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-sample_submission.csv')
train.shape, test.shape, submit.shape

((12120, 6), (5195, 5), (5195, 2))

In [6]:
train.head()
# XLM-RoBERTa 쓰기 딱 좋구만...

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [3]:
df = shuffle(train)
sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_train = df['label']

fold_list = list(sk.split(df, y_train))

train_list = []
val_list = []

for i, fold in enumerate(fold_list):
  df_train = df[df.index.isin(fold[0])]
  df_val = df[df.index.isin(fold[1])]

  train_list.append(df_train)
  val_list.append(df_val)

print(f'train length : {len(train_list)}')
print(f'val length : {len(val_list)}')

train length : 5
val length : 5


In [3]:
# 위에 하고 같음

df = shuffle(train)
sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_train = df['label']

fold_list = list(sk.split(df, y_train))

list_train = []
list_val = []
for i, (train_index, val_index) in enumerate(sk.split(df, y_train)):
  train_df, val_df = df.iloc[train_index], df.iloc[val_index]

  list_train.append(train_df)
  list_val.append(val_df)

In [6]:
list_train[0]

,id,premise,hypothesis,lang_abv,language,label
1323,8ec5be35b1,He watched the river flow.,The riverbed was completely dry.,en,English,2
2601,917daf858f,I put it to you that you did do so?,I am guessing that you did stay at the hotel?,en,English,1
1355,147267c1a9,although the uh it's uh it we almost one day w...,We painted the house over the duration of one ...,en,English,0
270,4241535fa7,you know we keep a couple hundred dollars um i...,We have some money on there,en,English,0
1088,865a383c45,"Designed by George Meikle Kemp, an unknown dra...",The design was completely original and uncopied.,en,English,2
...,...,...,...,...,...,...
8033,3cf10465e1,Trại nào đúng là có hậu quả sức khỏe cộng đồng...,Tất cả các trại đều có hậu quả sức khỏe cộng đ...,vi,Vietnamese,1
10516,4729fc742c,At Kansas City Power and Light's Hawthorn Powe...,Unit 5 was the only one that was replaced.,en,English,1
145,a5484dd197,The finest is the huge conical-roofed Tomb/Pil...,The Tomb/Pillar of Absalom has a large cone-sh...,en,English,0
8612,35305767c9,"The fancifully decorated Macau Palace, a float...",Macau Palace is found on the eastern waterfront.,en,English,2


In [7]:
list_val[0]

,id,premise,hypothesis,lang_abv,language,label
2194,41ce3f8a0b,49 Bima ya mshahara inawezesha viparara wa sia...,wadanganyifu hufanya mambo yasiyokubalika na h...,sw,Swahili,1
9540,09b4fcf9d0,"Хотя я и сейчас не понимаю, как он вообще мог ...","Я не понимаю, почему он думал, что я это доделаю.",ru,Russian,0
4971,d960aad98e,Bill Clinton has developed a rhetoric and a se...,Bill Clinton is aware of the current divide an...,en,English,0
5008,8395afc7c5,189 и другие затраты оцениваются аналогичною,"Они предполагали, что издержки использования с...",ru,Russian,1
5310,a97a31ea73,Very simply.,In a complicated way.,en,English,2
...,...,...,...,...,...,...
10146,ec6c1214ca,Inside the Oval White House Tapes From FDR to...,Many tapes were taken in the white house,en,English,0
1426,21abde9983,Off El Hurriya Street you'll find the Neo-Clas...,"Interestingly enough, every item is a vampire ...",en,English,1
140,e7bc15b1b9,that's hilarious to to get that jack off that'...,I will enjoy telling that story.,en,English,1
401,3328cdd6d6,The information provided in this guide is curr...,The information in the guide is up-to-date.,en,English,0


# 토큰화, Dataset, Dataloader

In [11]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 1e-5
MAX_LEN = 256
NUM_EPOCHS = 3
BATCH_SIZE = 32

In [12]:
device = xm.xla_device()
print(device)

xla:1


In [13]:
# bert-base-multilingual-uncased 토큰
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE,
                                          do_lower_case=True)

In [15]:
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']
    sentence3 = self.df_data.loc[index, 'language']

    encoded_dict = tokenizer.encode_plus((sentence1, sentence2, sentence3),
                                             add_special_tokens=True,
                                             max_length=MAX_LEN,
                                             truncation=True,
                                             pad_to_max_length=True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]
    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)
    return sample

  def __len__(self):
    return len(df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df
  
  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']
    sentence3 = self.df_data.loc[index, 'language']

    encoded_dict = tokenizer.encode_plus((sentence1, sentence2, sentence3),
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, token_type_ids)
    return sample
  
  def __len__(self):
    return len(df_data)

In [16]:
# Dataset으로 tokenizer까지
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)
    return sample

  def __len__(self):
    return len(self.df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df



  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, tokens_type_ids)

    return sample

  def __len__(self):
    return len(self.df_data)

In [17]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

NameError: ignored

In [ ]:
# num_workers 쓰기 위한 cpu_count()
NUM_CORES = os.cpu_count()

NUM_CORES

40

In [ ]:
train = CompDataset(df_train)
valid = CompDataset(df_val)
test = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)

val_dataloader = torch.utils.data.DataLoader(valid,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=4)

test_dataloader = torch.utils.data.DataLoader(test,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=4)

len(train_dataloader), len(val_dataloader), len(test_dataloader)

(303, 76, 163)

# Define the Model(BertForSequenceClassification)

In [ ]:
# model 불러오기(BertForSequnceClassification)
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

# Test the Model

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=8,
                                               shuffle=True,
                                               num_workers=1)

batch = next(iter(train_dataloader))

b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_token_type_ids = batch[2].to(device)
b_labels = batch[3].to(device)

b_input_ids.size(), b_input_mask.size(), b_token_type_ids.size(), b_labels.size()

(torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8]))

# 모델의 output 탐구



In [ ]:
# output 생성
outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask,
                labels=b_labels)

In [ ]:
 # 하나의 튜플 안에 (loss, predicts)
 len(outputs)

2

In [ ]:
# loss
outputs[0]

tensor(1.0701, device='xla:1', grad_fn=<NllLossBackward>)

In [ ]:
# predicts
outputs[1]

tensor([[ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493],
        [ 0.1001, -0.0112,  0.2493]], device='xla:1', grad_fn=<AddmmBackward>)

In [ ]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

array([0, 0, 0, 0, 2, 0, 0, 0])

In [ ]:
acc = accuracy_score(y_true=y_true, y_pred=y_pred)

acc

0.25

# Train the Model

In [27]:
import random
import gc

seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
  fold_val_acc_list.append([])

for epoch in range(0, NUM_FOLDS):
  print('\nNum folds used for training', NUM_FOLDS_TO_TRAIN)
  print(f'========= Epoch {epoch+1} / {NUM_EPOCHS} =========')

  num_folds = len(list_train)

  epoch_acc_score_list = []

  for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    print('\n== Fold Model', fold_index)

    if epoch == 0:
      model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                            num_labels=3,
                                                            output_attentions=False,
                                                            output_hidden_states=False)
      model.to(device)
      optimizer = AdamW(model.parameters(), lr=L_RATE)

    else:
      path_model = 'model_' + str(fold_index) + '.bin'
      model.load_state_dict(torch.load(path_model))
      model.to(device)
    
    df_train = list_train[fold_index]
    df_val = list_val[fold_index]

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)

    train_data = CompDataset(df_train)
    val_data = CompDataset(df_val)

    train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val_data,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)
    
    stacked_val_labels = []
    targets_list = []

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0

    for i, (batch) in enumerate(train_dataloader):
      input_ids = batch[0].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)
      target = batch[3].to(device)

      model.zero_grad()
      
      output = model(input_ids, 
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=target)
      
      loss = output[0]
      total_train_loss = total_train_loss + loss.item()

      optimizer.zero_grad()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      xm.optimizer_step(optimizer=optimizer, barrier=True)

    print(f'train loss : {total_train_loss/NUM_FOLDS_TO_TRAIN}')

    print('\nValidation')
    model.eval()

    torch.set_grad_enabled(False)

    total_val_loss=0

    for j, val_batch in enumerate(val_dataloader):
      input_ids = val_batch[0].to(device)
      attention_mask = val_batch[1].to(device)
      token_type_ids = val_batch[2].to(device)
      target = val_batch[3].to(device)

      output = model(input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids,
                     labels=target)
      
      loss = output[0]
      total_val_loss = total_val_loss + loss.item()

      preds = output[1]

      val_preds = preds.detach().cpu().numpy()
      targets_np = target.to('cpu').numpy()

      targets_list.extend(targets_np)

      if j == 0:
        stacked_val_preds = val_preds

      else:
        stacked_val_preds = np.vstack((stacked_val_preds,
                                       val_preds))
    
    # calculate val acc for this fold
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)

    epoch_acc_score_list.append(val_acc)

    print('Val loss:', total_val_loss)
    print('Val Acc:', val_acc)

    if epoch == 0:
      model_name = 'model_' + str(fold_index) + '.bin'
      torch.save(model.state_dict(), model_name)
      print('Saved model as', model_name)

    if epoch != 0:
      val_acc_list = fold_val_acc_list[fold_index]
      best_val_acc = max(val_acc_list)

      if val_acc > best_val_acc:
        model_name = 'model_' + str(fold_index) + '.bin'
        torch.save(model.state_dict(), model_name)
        print('Val acc improved, saved model...')

    fold_val_acc_list[fold_index].append(val_acc)
    
    gc.collect()

  cv_acc = sum(epoch_acc_score_list) / NUM_FOLDS_TO_TRAIN
  print('\nCV Acc:', cv_acc)


Num folds used for training 3
========= Epoch 1 / 3 =========

== Fold Model 0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
train loss : 101.10404324531555

Validation
Val loss: 66.89265096187592
Val Acc: 0.5944719471947195
Saved model as model_0.bin

== Fold Model 1


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
train loss : 101.80440930525462

Validation
Val loss: 66.61745792627335
Val Acc: 0.6051980198019802
Saved model as model_1.bin

== Fold Model 2


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
train loss : 101.5918397307396

Validation
Val loss: 65.6192399263382
Val Acc: 0.6126237623762376
Saved model as model_2.bin

CV Acc: 0.6040979097909791

Num folds used for training 3
========= Epoch 2 / 3 =========

== Fold Model 0
Training...
train loss : 81.13626380761464

Validation
Val loss: 63.00364923477173
Val Acc: 0.641914191419142
Val acc improved, saved model...

== Fold Model 1
Training...
train loss : 82.57431001464526

Validation
Val loss: 61.53683412075043
Val Acc: 0.6427392739273927
Val acc improved, saved model...

== Fold Model 2
Training...
train loss : 81.24642755587895

Validation
Val loss: 61.14067977666855
Val Acc: 0.6373762376237624
Val acc improved, saved model...

CV Acc: 0.6406765676567657

Num folds used for training 3
========= Epoch 3 / 3 =========

== Fold Model 0
Training...
train loss : 65.97585199276607

Validation
Val loss: 63.75871032476425
Val Acc: 0.6468646864686468
Val acc improved, saved model...

== Fold Model 1
Training...
train los

In [ ]:
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



# Store the accuracy scores for each fold model in this list.
# [[model_0 scores], [model_1 scores], [model_2 scores], [model_3 scores], [model_4 scores]]
# [[ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...]]

# Create a list of lists to store the val acc results.
# The number of items in this list will correspond to
# the number of folds that the model is being trained on.
fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
    
    # append an empty list
    fold_val_acc_list.append([])
    
    
    
    

# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    # Get the number of folds
    num_folds = len(train_list)

    # For this epoch, store the val acc scores for each fold in this list.
    # We will use this list to calculate the cv at the end of the epoch.
    epoch_acc_scores_list = []
    
    # For each fold...
    for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
        
        print('\n== Fold Model', fold_index)
        
        
        # .........................
        # Load the fold model
        # .........................
        
        if epoch == 0:
            
            # define the model
            model = BertForSequenceClassification.from_pretrained(
            MODEL_TYPE, 
            num_labels = 3,       
            output_attentions = False, 
            output_hidden_states = False,
            )
            
            # Send the model to the device.
            model.to(device)
            
            optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8
            )
            
        else:
        
            # Get the fold model
            path_model = 'model_' + str(fold_index) + '.bin'
            model.load_state_dict(torch.load(path_model))

            # Send the model to the device.
            model.to(device)
        
        
        
        # .....................................
        # Set up the train and val dataloaders
        # .....................................
        
        
        # Intialize the fold dataframes
        df_train = train_list[fold_index]
        df_val = val_list[fold_index]
        
        # Reset the indices or the dataloader won't work.
        df_train = df_train.reset_index(drop=True)
        df_val = df_val.reset_index(drop=True)
    
        # Create the dataloaders
        train_data = CompDataset(df_train)
        val_data = CompDataset(df_val)

        train_dataloader = torch.utils.data.DataLoader(train_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=4)

        val_dataloader = torch.utils.data.DataLoader(val_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=4)
    
    
    

       

        # ========================================
        #               Training
        # ========================================
        
        stacked_val_labels = []
        targets_list = []

        print('Training...')

        # put the model into train mode
        model.train()

        # This turns gradient calculations on and off.
        torch.set_grad_enabled(True)


        # Reset the total loss for this epoch.
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):

            train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

            print(train_status, end='\r')


            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_token_type_ids = batch[2].to(device)
            b_labels = batch[3].to(device)

            model.zero_grad()        


            outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask,
                        labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_train_loss = total_train_loss + loss.item()

            # Zero the gradients
            optimizer.zero_grad()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Use the optimizer to update Weights
            
            # Optimizer for GPU
            # optimizer.step() 
            
            # Optimizer for TPU
            # https://pytorch.org/xla/
            xm.optimizer_step(optimizer, barrier=True)
            
           


        print('Train loss:' ,total_train_loss)


        # ========================================
        #               Validation
        # ========================================

        print('\nValidation...')

        # Put the model in evaluation mode.
        model.eval()

        # Turn off the gradient calculations.
        # This tells the model not to compute or store gradients.
        # This step saves memory and speeds up validation.
        torch.set_grad_enabled(False)


        # Reset the total loss for this epoch.
        total_val_loss = 0


        for j, val_batch in enumerate(val_dataloader):

            val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

            print(val_status, end='\r')

            b_input_ids = val_batch[0].to(device)
            b_input_mask = val_batch[1].to(device)
            b_token_type_ids = val_batch[2].to(device)
            b_labels = val_batch[3].to(device)      


            outputs = model(b_input_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()

            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
                
                
                
        # .........................................
        # Calculate the val accuracy for this fold
        # .........................................      


        # Calculate the validation accuracy
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)
        
        
        epoch_acc_scores_list.append(val_acc)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)
        
        
        # .........................
        # Save the best model
        # .........................
        
        if epoch == 0:
            
            # Save the Model
            model_name = 'model_' + str(fold_index) + '.bin'
            torch.save(model.state_dict(), model_name)
            print('Saved model as ', model_name)
            
        if epoch != 0:
        
            val_acc_list = fold_val_acc_list[fold_index]
            best_val_acc = max(val_acc_list)
            
            if val_acc > best_val_acc:
                # save the model
                model_name = 'model_' + str(fold_index) + '.bin'
                torch.save(model.state_dict(), model_name)
                print('Val acc improved. Saved model as ', model_name)

            
                
                
                
        # .....................................
        # Save the val_acc for this fold model
        # .....................................
        
        # Note: Don't do this before the above 'Save Model' code or 
        # the save model code won't work. This is because the best_val_acc will
        # become current val accuracy.
                
        # fold_val_acc_list is a list of lists.
        # Each fold model has it's own list corresponding to the fold index.
        # Here we choose a list corresponding to the fold number and append the acc score to that list.
        fold_val_acc_list[fold_index].append(val_acc)
        
            

        # Use the garbage collector to save memory.
        gc.collect()


    
        
        
    # .............................................................
    # Calculate the CV accuracy score over all folds in this epoch
    # .............................................................   
        
        
    # Print the average val accuracy for all 5 folds
    cv_acc = sum(epoch_acc_scores_list)/NUM_FOLDS_TO_TRAIN
    print("\nCV Acc:", cv_acc)
    

In [ ]:
# Test Set

print('\nTest Set...')

model_preds_list = []

print('Total batches:', len(test_dataloader))

for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
  print('\nFold Model', fold_index)

  # load the fold model
  path_model = 'model_' + str(fold_index) + '.bin'
  model.load_state_dict(torch.load(path_model))

  model.to(device)

  stacked_val_labels = []

  model.eval()

  torch.set_grad_enabled(False)

  total_val_loss = 0

  for j, h_batch in enumerate(test_dataloader):
    infernce_status = 'Batch' + str(j+1)

    print(infernce_status, end='\r')

    b_input_ids = h_batch[0].to(device)
    b_input_mask = h_batch[1].to(device)
    b_token_type_ids = h_batch[2].to(device)

    outputs = model(b_input_ids,
                    token_type_ids = b_token_type_ids,
                    attention_mask = b_input_mask)
    
    preds = outputs[0]

    val_preds = preds.detach().cpu().numpy()

    # Stack the predictions

    if j == 0:
      stacked_val_preds = val_preds

    else:
      stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

  model_preds_list.append(stacked_val_preds)

In [ ]:
for i, item in enumerate(model_preds_list):
  if i == 0:
    preds = item

  else:
    preds = item+preds

avg_preds = preds / len(model_preds_list)

test_preds = np.argmax(avg_preds, axis=1)

test_preds

In [ ]:
# Submission
submit['prediction'] = test_preds
submit

# XLM-RoBERTa Model 사용

In [4]:
model_type = 'xlm-roberta-base'

L_RATE = 1e-8
MAX_LEN = 256

NUM_EPOCHS = 3
BATCH_SIZE = 32

In [5]:
device = xm.xla_device()
print(device)

xla:1


# Load the data

In [7]:
tokenizer = XLMRobertaTokenizer.from_pretrained(model_type)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

# Create the DataLoader

In [47]:
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0] 
    target = torch.tensor(self.df_data.loc[index, 'label'])
    
    sample = (input_ids, attention_mask, target)
    return sample

  def __len__(self):
    return len(self.df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]

    sample = (input_ids, attention_mask)
    return sample

  def __len__(self):
    return len(self.df_data)

In [48]:
train_data = CompDataset(df_train)
val_data = CompDataset(df_val)
test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_data,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=2)

print(len(train_dataloader), len(val_dataloader), len(test_dataloader))

303 76 163


# Define the Model(xlm-roberta for sequence classification)

In [11]:
model = XLMRobertaForSequenceClassification.from_pretrained(model_type,
                                                            num_labels=3)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [12]:
optimizer = AdamW(model.parameters(),
                  lr=L_RATE,
                  eps=1e-8)

# Train Model

In [49]:
import random
import gc

seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch in range(0, NUM_EPOCHS):
  print(f'========= Epoch {epoch+1} / {NUM_EPOCHS} =========')
    
  stacked_val_labels = []
  targets_list = []

  print('Training...')
  model.train()
  torch.set_grad_enabled(True)

  total_train_loss = 0

  for i, batch in enumerate(train_dataloader):
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    target = batch[2].to(device)

    model.zero_grad()
      
    output = model(input_ids, 
                    attention_mask=attention_mask,
                    labels=target)
      
    loss = output[0]
    total_train_loss = total_train_loss + loss.item()

    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    xm.optimizer_step(optimizer=optimizer, barrier=True)

  print(f'train loss : {total_train_loss}')

  print('\nValidation')
  model.eval()

  torch.set_grad_enabled(False)

  total_val_loss=0

  for j, val_batch in enumerate(val_dataloader):
    b_input_ids = val_batch[0].to(device)
    b_input_mask  = val_batch[1].to(device)
    b_labels  = val_batch[2].to(device)

    output = model(b_input_ids,
                    attention_mask=b_input_mask,
                    labels=b_labels)
      
    loss = output[0]
    total_val_loss = total_val_loss + loss.item()

    preds = output[1]

    val_preds = preds.detach().cpu().numpy()
    targets_np = b_labels.to('cpu').numpy()

    targets_list.extend(targets_np)

    if j == 0:
      stacked_val_preds = val_preds

    else:
        stacked_val_preds = np.vstack((stacked_val_preds,
                                       val_preds))
    
    # calculate val acc for this fold
  y_true = targets_list
  y_pred = np.argmax(stacked_val_preds, axis=1)

  val_acc = accuracy_score(y_true, y_pred)

  print('Val loss:', total_val_loss)
  print('Val Acc:', val_acc)
    
  torch.save(model.parameters, 'model.pt')
  gc.collect()

========= Epoch 1 / 3 =========
Training...
train loss : 334.8550956249237

Validation
Val loss: 83.71893048286438
Val Acc: 0.327970297029703
========= Epoch 2 / 3 =========
Training...
train loss : 335.2887850999832

Validation
Val loss: 83.70977783203125
Val Acc: 0.327970297029703
========= Epoch 3 / 3 =========
Training...
train loss : 335.32937812805176

Validation
Val loss: 83.7236179113388
Val Acc: 0.327970297029703


In [50]:
for j, batch in enumerate(test_dataloader):
  input_ids = batch[0].to(device)
  attention_mask = batch[1].to(device)

  outputs = model(input_ids,
                  attention_mask = attention_mask)
  
  preds = outputs[0]

  preds = preds.detach().cpu().numpy()
  
  if j == 0:
    stacked_preds = preds
  else:
    stacked_preds = np.vstack((stacked_preds, preds))

In [51]:
preds = np.argmax(stacked_preds, axis=1)
preds

array([1, 1, 1, ..., 1, 1, 1])

In [46]:
submit['prediction'] = preds
submit

,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,1
3,58518c10ba,1
4,c32b0d16df,1
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,1


# XLM-roBERTa train 필사(필사)
# https://www.kaggle.com/vbookshelf/basics-of-bert-and-xlm-roberta-pytorch#Section-3

# 자연어 처리의 4가지 단계
자연어 처리 솔루션을 만들기 위한 방법론은 정말 다양하지만, 대부분의 솔루션은 아래의 4가지 단계로 구성되어 있습니다. 이 포스트에서는 각각의 솔루션에 대해 소개하기보다 각 단계에 활용되는 주요 개념들을 다루어보았습니다.

  - Preprocessing : 
대부분의 자연어 데이터에는 URL과 같은 hypertext 및 특수문자들이 섞여있습니다. 특히 웹크롤링을 통해 데이터를 수집한 경우 각 웹페이지에 있는 텍스트 데이터는 모두 html 태그들과 섞여있을 것입니다. 따라서 자연어 데이터를 머신러닝, 딥러닝과 같은 통계 기법을 이용해 바로 분석하기에 앞서, 우선 분석하고자 하는 데이터에 사람들이 일상적으로 사용하는 언어만 남아있도록 가공해주어야 합니다.

   - 이 뿐 아니라 동일한 의미를 가지지만 축약하여 표현하여 그 형태가 달라보이거나(do not = don’t), 실제 문장의 의미 해석에 있어 크게 중요하지 않은 표현들(for, to, the) 또한 처리해주는 편이 좋습니다. 이 밖에도 정말 다양한 형태의 전처리를 수행할 수 있는데요, MBTI classification 관련 포스트에서 구체적인 예시를 찾아보실 수 있습니다.

  - Tokenization : 
데이터의 전처리가 끝나 마침내 자연어 데이터만 남았다면, 이제 문자열(string)을 다차원 벡터(vector)로 변환해주어야 합니다. 이렇게 변환한 벡터들이 위치한 공간을 임베딩(embedding)이라고 부르는데, 각 단어를 벡터로 변환하는 경우 단어 임베딩(word embedding), 각 문장을 벡터로 변환하는 경우 문장 임베딩(sentence embedding) 등으로 벡터화(vectorization)의 단위를 붙여 지칭하기도 합니다.

  - 단어 임베딩이란 앞서 말씀드린 바와 같이 말뭉치(corpus)에 포함되어 있는 단어들이 각각 하나의 좌표를 가지도록 형성한 벡터공간을 말합니다. 여기서 단어(word)란 일반적으로 띄어쓰기나 줄바꿈과 같은 공백 문자(whitespace)로 나뉘어져 있는 문자열의 일부분을 지칭합니다.

  Lorem ipsum dolor sit amet, …

  - 즉 위의 예시에서는 띄어쓰기로 구분되어 있는 Lorem, ipsum, dolor, sit, amet이 각각 하나의 단어가 됩니다. 그러나 띄어쓰기를 활용하지 않는 언어들의 경우, 위와 같은 정의를 통해서는 문자열을 여러 단어로 잘라낼 수 없습니다.

  努輸強紙暮革人士績質宿福読相理毎阪釣方新, …

  - 이 경우 공백 문자가 아닌 형태소와 같은 의미론적 단위로 문자열을 구분하거나, 여러 말뭉치(corpus)에서 자주 등장하는 글자(character)들의 순서쌍들을 묶어서 벡터공간을 형성하기도 합니다. 이 임베딩의 단위를 토큰(token)이라고 하고, 주어진 문자열을 토큰들로 나누는 과정을 토큰화(tokenization)이라고 부릅니다. 그리고 이러한 토큰들이 위치한 벡터공간을 토큰 임베딩(token embedding)이라고 합니다.

  - 참고로 한글 문자열의 자연어 처리에는 아래 예시와 같이 형태소(morpheme)를 이용한 임베딩을 자주 활용하고 있습니다.

  대한민국은 민주공화국이다.

  대한민국 + -은 + 민주공화국 + -이 + -다

  - Token Embedding : 
문자열을 단어, 형태소 등의 토큰으로 분해하는 데 성공했다면 우선 one-hot encoding을 통해 곧바로 각각의 토큰들을 벡터로 변환할 수 있습니다. One-hot encoding이란 말뭉치에 들어있는 단어들을 중복없이 순서대로 나열한 뒤, 아래 이미지에서와 같이 영벡터(zero-vector)에 해당하는 순서(index)의 요소에만 1의 값을 부여하는 방법을 말합니다. 나열한 단어들에 중복이 없으므로 각각의 단어들은 모두 고유한 벡터값을 가지게 됩니다.
   - 그러나 one-hot encoding을 통해 얻은 임베딩에서는 각각의 토큰들이 서로 같은 거리에 위치합니다. 즉 두 토큰이 다르다는 사실만 알려줄 뿐 두 토큰이 서로 어떠한 관계를 가지는지는 알려주지 못하는 것이죠. 토큰화 과정(tokenization)과 one-hot encoding을 통해 토큰의 순서를 정했다면 이제 유사한 의미를 가지는 토큰끼리는 서로 가까이 당기고, 관련 없는 토큰끼리는 밀어냄으로써 보다 유의미한 임베딩을 얻을 수 있습니다.

  - Document Embedding
말뭉치의 토큰들을 벡터로 만들었다면 이제 머신러닝, 딥러닝 알고리즘을 사용할 준비가 거의 다 끝났습니다. 예를 들어 한 영화에 대한 평론이 전반적으로 긍정적인 내용을 담고 있는지, 부정적인 내용을 담고 있는지 알고 싶다면 해당 평론 안에 포함되어 있는 토큰들의 임베딩을 모두 더해 평균치를 구해볼 수 있겠습니다. 분포가설에 따르면 긍정적 의미를 가진 토큰들은 마찬가지로 긍정적 의미를 가진 토큰들 근처에 위치할 것이고, 부정적 의미를 가진 토큰들은 긍정적 의미를 가진 토큰들과는 멀리 분포할 것이기에 평균치가 이들 중 어느쪽에 더 가까운지 계산해보면 됩니다.

   - 그러나 개별 토큰들의 임베딩을 종합하는 것만으로는 해결할 수 없는 문제들도 있습니다. 한가지 예시로는 동음이의어와 다의어가 있습니다.

  1) 밥을 많이 먹어 배가 부르다.

  2) 사과보다 배가 더 시원하다.

  - 1에서의 ‘배’와 2에서의 ‘배’ 모두 동일한 형태를 가지지만 서로 다른 의미를 가지고 있습니다. 1번에서의 ‘배’가 과일이 아닌 신체 부위를 뜻한다는 것을 알기 위해서는 ‘밥’, ‘먹-‘ 등의 형태소가 같은 문장 내에 위치한다는 것을 알아야 합니다.

  - 한편 BERT5는 문장의 일부를 다른 단어로 대체하거나 제거한 뒤(mask) 원래의 문장을 복원하는 방식으로 학습합니다. 즉 
i
번째 토큰이 <MASK>라는 토큰으로 대체되었다고 하면 
P
(
w
i
|
w
1
,
…
,
w
i
−
1
,
w
i
+
1
,
…
,
w
n
−
1
)
의 값이 가장 높은 토큰을 찾아 
i
번째 토큰 자리에 다시 넣어줄 수 있도록 해당 조건부 확률을 학습하는 것입니다.

  - 언어모델을 학습시켰다면 이를 분해한 뒤 그 속에 있는 잠재 변수를(latent variable) 꺼내어 문장 또는 문서의 임베딩으로써 활용할 수 있습니다. 만약 학습한 언어모델이 multilayer perceptron과 softmax 함수를 이용해 문자열을 확률공간으로 사영했다면 softmax의 바로 이전 layer(penultimate layer)의 출력값을 임베딩으로 사용할 수 있습니다. 또한 Transformer7와 같이 각 layer별로 latent variable이 특별한 의미를 가지는 경우 (e.g. word pair, trigram) 각 layer의 출력값을 모두 합쳐 하나의 임베딩으로 활용하기도 합니다.

  - 이렇게 임베딩을 얻었다면 이제 마침내 풀고자 하는 문제(downstream task)에 맞추어 머신러닝 또는 딥러닝 알고리즘을 적용하면 됩니다. GPT, BERT와 같이 큰 용량의 언어모델을 활용하는 경우 언어모델의 학습 과정에서부터 딥러닝 알고리즘을 사용하므로, 임베딩을 얻은 후에도 동일한 딥러닝 알고리즘을 이용해 downstream task를 학습시키곤 합니다. 이 경우 임베딩을 얻기 위한 학습과정을 pretraining, 임베딩 이후의 추가적인 학습과정을 fine-tuning이라고 부릅니다.

  